# 简介

这是一款名为‘FunctionalAgentWithRetrieval检索增强小助手’的演示应用，展示了如何结合BaizhongSearch和ERNIEBot的functional agent来回答用户的专业知识问题。首先，应用通过BaizhongSearch在知识库中检索相关内容，然后评估检索内容与问题的相关度。如果内容与问题高度相关，大模型将采用检索增强方式回答问题；否则，大模型将调用工具列表中的工具回答用户的问题。这种设计不仅扩展了大模型的专业领域知识，还保持了其在领域知识之外的通用对话能力


# 检索增强功能介绍
大型语言模型（LLM）是基于大规模数据集训练的，但这些数据集不会包含用户的特定数据。检索增强生成（Retrieval-Augmented Generation, RAG）通过在生成过程中动态地结合用户的数据来解决这个问题。这不是通过改变LLM的训练数据来实现的，而是通过允许模型实时访问和利用用户的数据，以提供更定制化和与上下文更相关的回应。

在RAG中，用户的数据被加载并准备好用于查询或“索引”。用户查询作用于索引，筛选出最相关的上下文。然后，这个上下文和用户的查询一起发送给LLM，连同一个提示语，LLM随后提供回应。

<div align="center">
    <img src="https://github.com/PaddlePaddle/ERNIE-Bot-SDK/assets/137043369/3ae0b6a8-e818-47ee-a537-207aed1323ce" width="1000px">
</div>

**RAG内部的五个关键阶段**

RAG包含五个关键阶段，这些阶段将是您构建的任何更大应用程序的一部分。这些阶段包括：

**加载**：这指的是将用户的数据从其存储位置获取到处理流程中。

**索引**：这意味着创建一个允许查询数据的数据结构。对于LLM，这几乎总是意味着创建向量嵌入，即数据含义的数值表示，以及许多其他元数据策略，以便轻松准确地找到与上下文相关的数据。

**存储**：一旦数据被索引，用户会希望存储索引及任何其他元数据，以避免重新索引的需要。

**查询**：对于任何给定的索引策略，用户可以利用LLM和LlamaIndex数据结构进行多种查询，包括子查询、多步骤查询和混合策略。

**评估**：任何处理流程中的一个关键步骤是检查其相对于其他策略的有效性，或者当用户做出更改时的有效性。

构建流程如下：

# 1. 导入第三方库

主要是在导入一些必要的Python库和模块，以便实现functionalagentwithretrieval的功能。
+ erniebot: 这是导入erniebot模块的语句，erniebot是一个包含ERNIE Bot实现的主要代码库。
+ ERNIEBot: 实现ERNIE Bot的主要类，包含了实现对话功能的主要逻辑。
+ BaizhongSearch:实现百中的检索功能。
+ calculator_tool: 实现计算器功能的tool。
+ WholeMemory: 用于存储和管理代理的记忆的类。

这里使用EB_AGENT_ACCESS_TOKEN, 申请地址请参考[accessToken](https://aistudio.baidu.com/index/accessToken)

In [44]:
import os
from erniebot_agent.agents import FunctionalAgentWithRetrieval
from erniebot_agent.retrieval import BaizhongSearch
from erniebot_agent.memory.whole_memory import WholeMemory
from erniebot_agent.chat_models.erniebot import ERNIEBot
from erniebot_agent.tools.calculator_tool import CalculatorTool
import erniebot

os.environ["EB_AGENT_ACCESS_TOKEN"] = "your access token"

# 2. 预处理

## 2.1 百中检索
这里使用aistudio的access_token、knowledge_base_name、knowledge_base_id, 申请地址请参考[accessToken](https://aistudio.baidu.com/index/accessToken)

In [45]:
aistudio_access_token = os.environ.get("EB_AGENT_ACCESS_TOKEN", "")
baizhong_search = BaizhongSearch(
    access_token=aistudio_access_token,
    knowledge_base_name="test",
    knowledge_base_id="495735236530245",
)

以下代码是使用百中检索进行搜索的一个例子，流程大致如下：
1. 调用BaizhongSearch对象的search函数，并传入查询字符串"城市管理执法主管部门的职责是什么？"。
2. 将搜索结果以格式化的形式进行打印，这里使用了pprint模块进行美化打印。最终的结果存储在变量res中。

In [22]:
res = baizhong_search.search(query="城市管理执法主管部门的职责是什么？")
from pprint import pprint

pprint(res)

[{'content': '住房和城乡建设部规章城市管理执法办法（2017 年 1 月 24 日中华人民共和国住房和城乡建设部令第34 号公布 自 2017 '
             '年 5 月 1 日起施行）第一章 总 则第一条 '
             '为了规范城市管理执法工作，提高执法和服务水平，维护城市管理秩序，保护公民、法人和其他组织的合法权益，根据行政处罚法、行政强制法等法律法规的规定，制定本办法。第二条 '
             '城市、县人民政府所在地镇建成区内的城市管理执法活动以及执法监督活动，适用本办法。本办法所称城市管理执法，是指城市管理执法主管部门在城市管理领域根据法律法规规章规定履行行政处罚、行政强制等行政执法职责的行为。- '
             '1 -X住房和城乡建设部发布\x0c'
             '住房和城乡建设部规章第三条 '
             '城市管理执法应当遵循以人为本、依法治理、源头治理、权责一致、协调创新的原则，坚持严格规范公正文明执法。第四条 '
             '国务院住房城乡建设主管部门负责全国城市管理执法的指导监督协调工作。各省、自治区人民政府住房城乡建设主管部门负责本行政区域内城市管理执法的指导监督考核协调工作。城市、县人民政府城市管理执法主管部门负责本行政区域内的城市管理执法工作。第五条 '
             '城市管理执法主管部门应当推动建立城市管理协调机制，协调有关部门做好城市管理执法工作。第六条 '
             '城市管理执法主管部门应当加强城市管理法律法规规章的宣传普及工作，增强全民守法意识，共同维护城市管理秩序。第七条 '
             '城市管理执法主管部门应当积极为公众监督城市管理执法活动提供条件。- 2 -X住房和城乡建设部发布\x0c'
             '住房和城乡建设部规章第二章 执法范围第八条 '
             '城市管理执法的行政处罚权范围依照法律法规和国务院有关规定确定，包括住房城乡建设领域法律法规规章规定的行政处罚权，以及环境保护管理、工商管理、交通管理、水务管理、食品药品监管方面与城市管理相关部分的行政处罚权。第九条 '
             '需要集中行使的城市管理执

## 2.2 配置 ERNIE BOT API

In [46]:
erniebot.api_type = "aistudio"

# 3. Tool 构建
这段代码创建一个 CalculatorTool 类的实例，并将其赋值给变量 calculator_tool

In [47]:
calculator_tool = CalculatorTool()

# 4. FunctionalAgentwithRetrieval
FunctionalAgentwithRetrieval能够通过其检索增强功能回答用户关于专业知识的问题。

In [50]:
# 创建一个ERNIEBot实例，使用"ernie-4.0"模型。
llm = ERNIEBot(model="ernie-4.0")
# 创建一个WholeMemory实例。这是一个用于存储对话历史和上下文信息的类，有助于模型理解和持续对话。
memory = WholeMemory()
# 创建一个FunctionalAgent实例。这个代理将使用上面创建的ERNIEBot模型和WholeMemory，同时传入了一个名为calculator_tool的工具。
agent = FunctionalAgentWithRetrieval(
    llm=llm, tools=[calculator_tool], memory=memory, knowledge_base=baizhong_search, threshold=0.2
)

## 4.1 检索示例
在这个例子中，FunctionalAgentWithRetrieval使用其检索增强功能来回答用户的query

In [31]:
# 定义一个查询字符串，这个查询是关于"城乡建设部规章中，城市管理执法第三章，第十三条"的内容。
response = await agent._async_run("城乡建设部规章中，城市管理执法第三章，第十三条是什么？")
# 使用agent的async_run方法来异步执行查询。由于这是异步操作，因此需要使用'await'关键字。
messages = response.chat_history
for item in messages:
    print(item.to_dict())

{'role': 'user', 'content': '检索结果:\n\n    第1个段落: 住房和城乡建设部规章城市管理执法办法（2017 年 1 月 24 日中华人民共和国住房和城乡建设部令第34 号公布 自 2017 年 5 月 1 日起施行）第一章 总 则第一条 为了规范城市管理执法工作，提高执法和服务水平，维护城市管理秩序，保护公民、法人和其他组织的合法权益，根据行政处罚法、行政强制法等法律法规的规定，制定本办法。第二条 城市、县人民政府所在地镇建成区内的城市管理执法活动以及执法监督活动，适用本办法。本办法所称城市管理执法，是指城市管理执法主管部门在城市管理领域根据法律法规规章规定履行行政处罚、行政强制等行政执法职责的行为。- 1 -X住房和城乡建设部发布\x0c住房和城乡建设部规章第三条 城市管理执法应当遵循以人为本、依法治理、源头治理、权责一致、协调创新的原则，坚持严格规范公正文明执法。第四条 国务院住房城乡建设主管部门负责全国城市管理执法的指导监督协调工作。各省、自治区人民政府住房城乡建设主管部门负责本行政区域内城市管理执法的指导监督考核协调工作。城市、县人民政府城市管理执法主管部门负责本行政区域内的城市管理执法工作。第五条 城市管理执法主管部门应当推动建立城市管理协调机制，协调有关部门做好城市管理执法工作。第六条 城市管理执法主管部门应当加强城市管理法律法规规章的宣传普及工作，增强全民守法意识，共同维护城市管理秩序。第七条 城市管理执法主管部门应当积极为公众监督城市管理执法活动提供条件。- 2 -X住房和城乡建设部发布\x0c住房和城乡建设部规章第二章 执法范围第八条 城市管理执法的行政处罚权范围依照法律法规和国务院有关规定确定，包括住房城乡建设领域法律法规规章规定的行政处罚权，以及环境保护管理、工商管理、交通管理、水务管理、食品药品监管方面与城市管理相关部分的行政处罚权。第九条 需要集中行使的城市管理执法事项，应当同时具备下列条件：（一）与城市管理密切相关；（二）与群众生产生活密切相关、多头执法扰民问题突出；（三）执法频率高、专业技术要求适宜；（四）确实需要集中行使的。第十条 城市管理执法主管部门依法相对集中行使行政处罚权的，可以实施法律法规规定的与行政处罚权相关的行政强制措施。- 3 -X住房和城乡建设部发布\x0c住房和城乡建设部规章第十

## 4.2 调用tool示例
在这个例子中，FunctionalAgentWithRetrieval没有利用其检索信息，而是直接调用计算器工具回答用户的问题

In [51]:
# 定义一个查询字符串，这个查询是关于"11-2="的内容。
response = await agent._async_run("11-2=")
# 使用agent的async_run方法来异步执行查询。由于这是异步操作，因此需要使用'await'关键字。
messages = response.chat_history
for item in messages:
    print(item.to_dict())

{'role': 'user', 'content': '11-2='}
{'role': 'assistant', 'content': '', 'function_call': {'name': 'CalculatorTool', 'thoughts': "用户想知道11减2等于多少，我可以使用CalculatorTool工具来计算公式，其中`math_formula`字段的内容为：'11-2'。", 'arguments': '{"math_formula":"11-2"}'}, 'search_info': None}
{'role': 'function', 'name': 'CalculatorTool', 'content': '{"formula_result": 9}'}
{'role': 'assistant', 'content': '根据你的请求，11减2的结果为：9。如果你还有其他问题，请随时告诉我。', 'function_call': None, 'search_info': None}
